In [1]:
import random
import numpy as np
from radcad import Model, Simulation, Experiment
from models import Ticket, TicketHolderAgent # self defined in models.py

#  Define Parameters
params = {
    'selling_mechanism': 'EIP-1559',  # Change to 'first_price' to switch mechanism
    'max_tickets': 100,
    'ticket_price': 10,
    'MEV_scale': 30,
    'slots_per_epoch': 32,
    'number_of_ticket_holders': 10,
}

'''
class Ticket:
    def __init__(self, id):
        self.id = id
        self.assigned = False
        self.holder_id = None
        self.assigned_slot = None
        self.assigned_epoch = None
        self.price_paid = 0 
        self.redeemed = False

# Define the TicketHolders as Agents
class TicketHolderAgent:
    def __init__(self, id, funds):
        self.id = id
        self.tickets = []
        self.available_funds = funds
        self.intrinsic_valuation = random.uniform(10, 50)
        self.MEV_capture_rate = random.uniform(0.1, 1.0)
        self.aggressiveness = random.uniform(0.01, 0.3) # to be adjusted later
        self.discount_factor = 1 # to be adjusted later

    def decide_bid_first_price(self): # Needs to include which ticket to buy, slot information etc? All in self?
        """Calculate bid based on intrinsic valuation, funds, and aggressiveness."""
        max_bid = self.intrinsic_valuation * (1 - self.aggressiveness)
        return min(max_bid, self.available_funds)
'''

# State Update Functions
def setup_initial_state(params):
    tickets = [Ticket(i+1) for i in range(params['max_tickets'])]
    agents = [TicketHolderAgent(i, random.uniform(100, 500)) for i in range(params['number_of_ticket_holders'])]
    return {'tickets': tickets, 
            'ticket_holder': agents, 
            'current_ticket_id': params['max_tickets'], 
            'ticket_price': params['ticket_price'],
            'MEV_per_slot': np.random.exponential(params['MEV_scale']),
            'total_MEV_captured': 0,
            'slots_to_tickets': {},
            'epoch': 0, 
            'slot': 0}

def ticket_issuance(state, params):
    print("Entering ticket_issuance with state:", state.keys())
    tickets = state['tickets']
    current_ticket_id = state['current_ticket_id']
    unredeemed_tickets = [t for t in tickets if not t.redeemed]
    if len(unredeemed_tickets) < params['max_tickets']:
        # Calculate how many new tickets to issue
        new_tickets_needed = params['max_tickets'] - len(unredeemed_tickets)
        for _ in range(new_tickets_needed):
            state['current_ticket_id'] += 1
            new_ticket = Ticket(state['current_ticket_id'])
            tickets.append(new_ticket)
        print(f"Issued {new_tickets_needed} new tickets.")
    return tickets, current_ticket_id

def assign_tickets_to_slots(state, params, tickets, epoch):   
    all_held_tickets = [
        ticket for holder in state['ticket_holders'] 
        for ticket in holder.tickets 
        if not ticket.redeemed
    ]
    random.shuffle(all_held_tickets)  # Randomize tickets to distribute slots fairly
    start_slot = epoch * params['slots_per_epoch'] + 1  # Ensure indexing starts correctly based on the epoch

    for i, ticket in enumerate(all_held_tickets[:params['slots_per_epoch']]):
        ticket['assigned_slot'] = start_slot + i
        ticket['assigned_epoch'] = epoch
        ticket['assigned'] = True
        
    print(f"Number of Assigned Tickets to Slots: {min(len(all_held_tickets), params['slots_per_epoch'])}")
    
    return state

def redeem_tickets(state, params):
    current_slot = state['slot']
    ticket_holders = state['ticket_holders']
    
    redeemable_ticket = next((ticket for ticket in state['tickets'] 
                              if ticket.get('assigned_slot') == current_slot and not ticket.get('redeemed', False)), None)

    if redeemable_ticket:
        ticket = redeemable_ticket
        ticket['redeemed'] = True
        
        # Find the ticket holder and calculate the MEV captured
        ticket_holder = next((holder for holder in ticket_holders if holder['id'] == ticket['holder_id']), None)
        if ticket_holder:
            mev_captured = params['MEV_per_slot'] * ticket_holder['MEV_capture_rate']
            ticket_holder['available_funds'] += mev_captured
            # Update the total MEV captured in the state
            state['total_MEV_captured'] += mev_captured
            print(f"Ticket {ticket['id']} redeemed by Ticket Holder {ticket_holder['id']}. MEV captured: {mev_captured:.2f}, Total Funds of Holder: {ticket_holder['available_funds']:.2f}")
        else:
            print(f"Ticket {ticket['id']} redeemed, but no ticket holder found.")
    else:
        print(f"No Tickets found for slot {current_slot}")

    return {'tickets': state['tickets'], 'total_mev_captured': state['total_MEV_captured']}


def assign_ticket_to_holder(state, holder, ticket):
    ticket.holder_id = holder.id
    ticket.assigned = True
    holder.available_funds -= state['ticket_price']
    
    print(f"Ticket {ticket.id} assigned to Holder {holder.id} at price {ticket_price:.3f}")


def purchase_tickets_policy(state, params):
    tickets_available = [t for t in state['tickets'] if t['holder_id'] is None]
    print(f"Tickets Available at Start: {len(tickets_available)}")  # Debug print

    ticket_holders = state['ticket_holders']
    ticket_price = state['ticket_price']
    
    if params['selling_mechanism'] == 'first_price':
        purchase_tickets_first_price(state, tickets_available)
    elif params['selling_mechanism'] == 'EIP-1559':
        purchase_tickets_EIP_1559(state, params, tickets_available)
    else:
        print("Mechanism not yet defined")
    return state

        
def purchase_tickets_first_price(state, tickets_available):
    if tickets_available:
        bids = [
            (holder, holder.decide_bid_first_price()) for holder in state['ticket_holders']
            if holder.available_funds >= state['ticket_price']
            ]
        if bids:
            # Find the highest bid
            max_bid = max(bids, key=lambda x: x[1])
            holder = max_bid[0]
            # Assign only one ticket to the highest bidder
            ticket = tickets_available.pop(0)
            assign_ticket_to_holder(state, holder, ticket, state['ticket_price'])
            # Update the ticket price to the highest bid
            state['ticket_price'] = max_bid[1]
    return state
 
def purchase_tickets_EIP_1559(state, params, tickets_available):
    
    # Randomize the order of ticket holders to ensure fairness
    random.shuffle(state['ticket_holders'])

    # Adjust ticket price as one ticket has been redeemed in the last round, only excluding the initial round
    if state['slot'] != 1:
        adjust_ticket_price_1559(state, params)
            
    # Prepare the queue of holders and the number of tickets they can potentially buy
    holders_queue = [[holder, min(holder.available_funds // state['ticket_price'], len(tickets_available))]
                        for holder in state['ticket_holders'] if holder.available_funds >= state['ticket_price']]
    
    # Flag to check if at least one ticket was purchased in the last full cycle
    tickets_purchased = True
    
    while tickets_available and tickets_purchased:
        tickets_purchased = False  # Reset the flag at the start of the cycle
    
        # Iterate over each holder to attempt ticket purchases
        for holder_data in holders_queue:
            holder, max_tickets = holder_data
    
            if max_tickets > 0:
                # Assign ticket and update holder data
                ticket = tickets_available.pop(0)
                assign_ticket_to_holder(state, holder, ticket, state['ticket_price'])
                holder_data[1] -= 1  # Decrement the tickets this holder can still buy
                tickets_purchased = True  # Set flag as successful purchase made
    
                # Adjust the ticket price after each ticket sale
                adjust_ticket_price_1559(state, params)
                    
                    # Re-evaluate the maximum number of tickets this holder can buy
                holder_data[1] = min(holder.available_funds // state['ticket_price'], len(tickets_available))
    
                if not tickets_available:  # Exit if no tickets are left
                    break
    
        # Remove holders who can no longer buy tickets
        holders_queue = [hd for hd in holders_queue if hd[1] > 0 and hd[0].available_funds >= self.ticket_price]
    
    if not tickets_available:
        print("All tickets sold.")
    elif not any(hd[1] > 0 for hd in holders_queue):
        print("No more buyers can afford tickets at the current price.")
  
def adjust_ticket_price_1559(state, params):
    if params['selling_mechanism'] == 'EIP-1559':
        total_tickets_held = sum(len([ticket for ticket in holder.tickets if not ticket.redeemed]) for holder in state['ticket_holders'])
        print(f"Total Tickets held: {total_tickets_held}")
        target_tickets = params['max_tickets']/2
        d = 8  # Adjustment factor
        state['ticket_price'] *= (1 + 1/d * ((total_tickets_held - target_tickets) / target_tickets))
        print(f"Adjusted Ticket Price: {state['ticket_price']:.2f}")

   
def update_market(state, substep, previous_state, policy_input, params):
    try:
        print("Attempting to access 'slot' in previous_state...")
        new_slot = previous_state['slot'] + 1
        # More logic...
    except TypeError as e:
        print("Caught TypeError:", e)
        print("previous_state content:", previous_state)
        print("Type of previous_state:", type(previous_state))
        raise  # Optionally re-raise to see the full traceback
    print(f"State after update: {state}")
    print (f"------- Beginning of Slot {new_slot} ----------")
    tickets, new_ticket_id = ticket_issuance(state, params) # tbd if this should be in variables
    
    if ((new_slot - 1) % params['slots_per_epoch'] == 0): # new Epoch
            new_epoch = state['epoch'] + 1
            state = assign_tickets_to_slots(state, params, tickets, new_epoch) #tbd on the return , needs later to be varaible
    else: 
        new_epoch = state['epoch']

    MEV_per_slot = np.random.exponential(params['MEV_scale'])

    return {'epoch': new_epoch, 'slot': new_slot, 'tickets': tickets, 'current_ticket_id': current_ticket_id,'MEV_per_slot': MEV_per_slot}



          
initial_state = setup_initial_state(params)
model = Model(
    params=params,
    initial_state=initial_state,
    state_update_blocks=[
        {
            'policies': {
                #test_policy
            },
            'variables': {
                #'epoch': update_market,
                'slot': update_market
                #'tickets': update_market,
                #'current_ticket_id': update_market,
                #'MEV_per_slot': update_market
            }
        },
        {
            'policies': {
                purchase_tickets_policy
                #'secondary_market': 
            },
            'variables': {
                'tickets': redeem_tickets,
                'total_MEV_captured': redeem_tickets
                #'statistics': collect_statistics # tbd if out of the box included
            }
        }
    ]
)

# Set up and run the simulation
simulation = Simulation(model=model, timesteps=100, runs=1)

results = simulation.run()

# Results output
print(f"Total MEV Captured: {state.total_MEV_captured}")
print(f"Final Ticket Price: {state.ticket_price}")

Attempting to access 'slot' in previous_state...
Caught TypeError: list indices must be integers or slices, not str
previous_state content: [[{'tickets': [<models.Ticket object at 0x113b56510>, <models.Ticket object at 0x113b56690>, <models.Ticket object at 0x113b56750>, <models.Ticket object at 0x113b56810>, <models.Ticket object at 0x113b568d0>, <models.Ticket object at 0x113b56990>, <models.Ticket object at 0x113b56a50>, <models.Ticket object at 0x113b56b10>, <models.Ticket object at 0x113b56bd0>, <models.Ticket object at 0x113b56c90>, <models.Ticket object at 0x113b56d50>, <models.Ticket object at 0x113b56e10>, <models.Ticket object at 0x113b56ed0>, <models.Ticket object at 0x113b56f90>, <models.Ticket object at 0x113b57050>, <models.Ticket object at 0x113b57110>, <models.Ticket object at 0x113b571d0>, <models.Ticket object at 0x113b57290>, <models.Ticket object at 0x113b57350>, <models.Ticket object at 0x113b57410>, <models.Ticket object at 0x113b574d0>, <models.Ticket object at 0

Simulation 0 / run 1 / subset 0 failed!
                Catching exception and returning partial results because option Engine.raise_exceptions == False.


TypeError: list indices must be integers or slices, not str

In [77]:
## TO DO's
# [] check that all variables are incldued
# [] write update total_MEV_captured function
# [] build secondary market
# [] more complex demand functions
# [] integrate pricing models
# []  make lookahead variable

In [5]:
import random
import numpy as np
from radcad import Model, Simulation, Experiment
from models import Ticket, TicketHolderAgent # self defined


#  Define Parameters
params = {
    'selling_mechanism': 'EIP-1559',  # Change to 'first_price' to switch mechanism
    'max_tickets': 100,
    'ticket_price': 10,
    'MEV_scale': 30,
    'slots_per_epoch': 32,
    'number_of_ticket_holders': 10,
}
'''
class Ticket:
    def __init__(self, id):
        self.id = id
        self.assigned = False
        self.holder_id = None
        self.assigned_slot = None
        self.assigned_epoch = None
        self.price_paid = 0 
        self.redeemed = False

# Define the TicketHolders as Agents
class TicketHolderAgent:
    def __init__(self, id, funds):
        self.id = id
        self.tickets = []
        self.available_funds = funds
        self.intrinsic_valuation = random.uniform(10, 50)
        self.MEV_capture_rate = random.uniform(0.1, 1.0)
        self.aggressiveness = random.uniform(0.01, 0.3) # to be adjusted later
        self.discount_factor = 1 # to be adjusted later

    def decide_bid_first_price(self): # Needs to include which ticket to buy, slot information etc? All in self?
        """Calculate bid based on intrinsic valuation, funds, and aggressiveness."""
        max_bid = self.intrinsic_valuation * (1 - self.aggressiveness)
        return min(max_bid, self.available_funds)

'''
def setup_initial_state(params):
    tickets = [Ticket(i+1) for i in range(params['max_tickets'])]
    agents = {}
    return {'tickets': tickets, 
            'ticket_holder': agents, 
            'current_ticket_id': params['max_tickets'], 
            'ticket_price': params['ticket_price'],
            'MEV_per_slot': np.random.exponential(params['MEV_scale']),
            'total_MEV_captured': 0,
            'slots_to_tickets': {},
            'epoch': 0, 
            'slot': 0}


def update_market(state, substep, previous_state, policy_input, params):
    return state



initial_state = setup_initial_state(params)
model = Model(
    params=params,
    initial_state=initial_state,
    state_update_blocks=[
        {
            'policies': {},
            'variables': {
                'epoch': update_market,
                'slot': update_market,
                'tickets': update_market,
                'current_ticket_id': update_market,
                'MEV_per_slot': update_market
            }
        },
        '''{
            'policies': {
                purchase_tickets_policy
                'secondary_market': 
            },
            'variables': {
                'tickets': redeem_tickets,
                'total_MEV_captured': redeem_tickets
                #'statistics': collect_statistics # tbd if out of the box included
            }
        }'''
    ]
)

# Set up and run the simulation
simulation = Simulation(model=model, timesteps=100, runs=1)

results = simulation.run()

# Results output
print(f"Total MEV Captured: {state.total_MEV_captured}")
print(f"Final Ticket Price: {state.ticket_price}")


ValueError: too many values to unpack (expected 2)